In [2]:
import duckdb
import pandas as pd
import os

In [6]:
df_covoit = duckdb.read_csv("../data/data_covoit/raw/nb-lieux-covoiturage_2025_export.csv")
df_covoit

┌───────┬──────────────────────────────┬─────────────────────┬────────┬─────────────┐
│ annee │             lieu             │      type_lieu      │ valeur │ territoryid │
│ int64 │           varchar            │       varchar       │ double │    int64    │
├───────┼──────────────────────────────┼─────────────────────┼────────┼─────────────┤
│  2025 │ Bordeaux Métropole           │ Supermarché         │    0.0 │   243300316 │
│  2025 │ Bordeaux Métropole           │ Sortie d'autoroute  │    0.0 │   243300316 │
│  2025 │ Bordeaux Métropole           │ Parking relais      │   24.0 │   243300316 │
│  2025 │ Bordeaux Métropole           │ Parking             │    0.0 │   243300316 │
│  2025 │ Bordeaux Métropole           │ Délaissé routier    │    0.0 │   243300316 │
│  2025 │ Bordeaux Métropole           │ Auto-stop           │    0.0 │   243300316 │
│  2025 │ Bordeaux Métropole           │ Aire de covoiturage │   41.0 │   243300316 │
│  2025 │ Brest Métropole              │ Supermarché  

In [9]:
query = """ 
SELECT 
    territoryid AS siren,
    sum(valeur) AS nb_aires_covoiturage
FROM df_covoit
WHERE type_lieu = 'Aire de covoiturage'
GROUP BY territoryid
"""

df_covoit_agg = duckdb.query(query)
df_covoit_agg

┌───────────┬──────────────────────┐
│   siren   │ nb_aires_covoiturage │
│   int64   │        double        │
├───────────┼──────────────────────┤
│ 200040277 │                  2.0 │
│ 200011773 │                  0.0 │
│ 200068997 │                  2.0 │
│ 243300563 │                  2.0 │
│ 246800726 │                  4.0 │
│ 243400769 │                  7.0 │
│ 248100430 │                  0.0 │
│ 200066876 │                  0.0 │
│ 200070993 │                  1.0 │
│ 200067965 │                  0.0 │
│     ·     │                   ·  │
│     ·     │                   ·  │
│     ·     │                   ·  │
│ 242500361 │                  6.0 │
│ 247200132 │                  2.0 │
│ 995329083 │                  0.0 │
│ 200046977 │                 35.0 │
│ 200030195 │                  0.0 │
│ 200023414 │                  7.0 │
│ 248300543 │                  3.0 │
│ 244500468 │                  3.0 │
│ 244200770 │                 15.0 │
│ 243100518 │                 12.0 │
├

In [13]:
df_epci = duckdb.read_csv("./data/raw/perimetre-epci-a-fp.csv", sep=";",ignore_errors=True)
df_epci

┌─────────┬───────────┬────────────────────────────────────┬──────────────────┬─────────────┬────────────┬───────────────┬───────────────┬─────────┬─────────┬──────────────┬────────────────────────┬───────────┬───────────┐
│  dept   │   siren   │           raison_sociale           │ nature_juridique │ mode_financ │ nb_membres │ total_pop_tot │ total_pop_mun │ dep_com │  insee  │ siren_membre │       nom_membre       │ ptot_2025 │ pmun_2025 │
│ varchar │   int64   │              varchar               │     varchar      │   varchar   │   int64    │    varchar    │    varchar    │ varchar │ varchar │    int64     │        varchar         │  varchar  │  varchar  │
├─────────┼───────────┼────────────────────────────────────┼──────────────────┼─────────────┼────────────┼───────────────┼───────────────┼─────────┼─────────┼──────────────┼────────────────────────┼───────────┼───────────┤
│ 01      │ 200029999 │ CC Rives de l'Ain - Pays du Cerdon │ CC               │ FPU         │         14 │ 1

In [ ]:
query_epci = """
SELECT 
    DISTINCT siren,
    dept,
    TRY_CAST(REPLACE(total_pop_tot,' ','') AS DOUBLE) AS total_pop_tot
FROM df_epci
"""

df_epci_filtered = duckdb.sql(query_epci)
df_epci_filtered

┌───────────┬─────────┬───────────────┬────────────┐
│   siren   │  dept   │ total_pop_tot │ nb_membres │
│   int64   │ varchar │    double     │   int64    │
├───────────┼─────────┼───────────────┼────────────┤
│ 200042497 │ 01      │       41975.0 │         19 │
│ 200070555 │ 01      │       23753.0 │         18 │
│ 240100610 │ 01      │       25906.0 │          9 │
│ 240100800 │ 01      │       25583.0 │          6 │
│ 240100891 │ 01      │       22090.0 │         12 │
│ 200040426 │ 02      │       16106.0 │         32 │
│ 200071892 │ 02      │       81012.0 │         39 │
│ 200071991 │ 02      │       29815.0 │         54 │
│ 200072031 │ 02      │       55209.0 │         87 │
│ 240200469 │ 02      │       14608.0 │         42 │
│     ·     │ ·       │          ·    │          · │
│     ·     │ ·       │          ·    │          · │
│     ·     │ ·       │          ·    │          · │
│ 249720061 │ 972     │      151772.0 │          4 │
│ 200050532 │ 976     │       30039.0 │       

In [15]:
query=""" 
SELECT 
    df_epci_filtered.siren,
    total_pop_tot,
    nb_aires_covoiturage,
    ROUND((1.0*nb_aires_covoiturage)/total_pop_tot*10000,3) AS aires_covoit_pour_10k_hab
FROM df_epci_filtered
LEFT JOIN df_covoit_agg
ON df_epci_filtered.siren = df_covoit_agg.siren
"""

df_covoit_relative = duckdb.sql(query)

In [18]:
df_nb_covoit = duckdb.read_csv("./data/raw/nb-trajets-covoiturage_2024_export.csv",sep=",")
df_nb_covoit

┌───────┬────────────────────────────────────────┬──────────┬─────────────┐
│ annee │                  lieu                  │  valeur  │ territoryid │
│ int64 │                varchar                 │  double  │    int64    │
├───────┼────────────────────────────────────────┼──────────┼─────────────┤
│  2024 │ Bordeaux Métropole                     │  68886.0 │   243300316 │
│  2024 │ Brest Métropole                        │  20089.0 │   242900314 │
│  2024 │ CA Agglo du Pays de Dreux              │  16937.0 │   200040277 │
│  2024 │ CA Agglomération d'Agen                │  11140.0 │   200096956 │
│  2024 │ CA Agglo Pays d'Issoire                │   4065.0 │   200070407 │
│  2024 │ CA Alès Agglomération                  │   8322.0 │   200066918 │
│  2024 │ CA Amiens Métropole                    │  19058.0 │   248000531 │
│  2024 │ CA Annemasse-Les Voirons-Agglomération │  19314.0 │   200011773 │
│  2024 │ CA Annonay Rhône Agglo                 │   3046.0 │   200072015 │
│  2024 │ CA

In [22]:
query = """ 
SELECT
    e.siren,
    total_pop_tot,
    valeur AS nb_trajets_covoiturage_2024,
    ROUND((1.0*valeur)/total_pop_tot*10000 ,3) AS nb_covoit_per_10k_hab
FROM df_epci_filtered e
LEFT JOIN df_nb_covoit d
ON d.territoryid = e.siren
"""

df_nb_covoit_relative = duckdb.sql(query)
df_nb_covoit_relative

┌───────────┬───────────────┬─────────────────────────────┬───────────────────────┐
│   siren   │ total_pop_tot │ nb_trajets_covoiturage_2024 │ nb_covoit_per_10k_hab │
│   int64   │    double     │           double            │        double         │
├───────────┼───────────────┼─────────────────────────────┼───────────────────────┤
│ 200070118 │       21470.0 │                       568.0 │               264.555 │
│ 240100750 │      106924.0 │                     16423.0 │              1535.951 │
│ 240200493 │       31401.0 │                       509.0 │               162.097 │
│ 200071082 │       60964.0 │                       939.0 │               154.025 │
│ 200071496 │       13941.0 │                       112.0 │                80.339 │
│ 200034700 │       65141.0 │                      9638.0 │               1479.56 │
│ 200071025 │       10226.0 │                        85.0 │                83.121 │
│ 200072304 │        7913.0 │                         9.0 │                1

In [25]:
query = """ 
SELECT 
    aires.*,
    covoit.nb_trajets_covoiturage_2024,
    covoit.nb_covoit_per_10k_hab
FROM df_covoit_relative AS aires
LEFT JOIN df_nb_covoit_relative AS covoit
ON aires.siren = covoit.siren
"""

df_covoit_final = duckdb.sql(query)

In [26]:
df_covoit_final.write_csv("./data/processed/covoiturage_epci.csv")